In [6]:
import yaml
from sqlalchemy import create_engine, inspect
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

yaml_file_path = 'db_creds.yaml'
db_connector = DatabaseConnector(yaml_file_path)

if db_connector.engine:
    tables = db_connector.list_db_tables()
    if tables:
        print("Tables in the Database:")
        for table in tables:
            print(table)
    else:
        print("Failed to list database tables.")
else:
    print("Failed to initialize the database engine.")


TypeError: DatabaseConnector.list_db_tables() missing 1 required positional argument: 'engine'

In [5]:
import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:
    def __init__(self, db_connector):
        self.db_connector = db_connector
        
    def read_rds_table(self, table_name):
        try:
            engine = self.db_connector.engine
            with engine.execution_options(isolation_level='AUTOCOMMIT').connect() as conn:    
                df = pd.read_sql(f"SELECT * FROM {table_name}", engine).set_index('index')
                return df
        
        except Exception as e:
            print(f"Error reading table {table_name}: {e}")
            return None

#table_name = "legacy_users"
#user_df = DataExtractor(db_connector).read_rds_table(table_name) 

In [3]:
legacy_user_df = user_df
legacy_user_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15320 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


In [16]:
duplicates = legacy_user_df.drop_duplicates()
print(duplicates.shape)

(15300, 11)


In [20]:
print(duplicates[duplicates.isin(['NULL']).any(axis=1)])

      first_name last_name date_of_birth company email_address address  \
index                                                                    
867         NULL      NULL          NULL    NULL          NULL    NULL   

      country country_code phone_number join_date user_uuid  
index                                                        
867      NULL         NULL         NULL      NULL      NULL  


In [17]:
legacy_user_df = legacy_user_df[~legacy_user_df.isin(['NULL']).any(axis=1)]
legacy_user_df.shape

(15299, 11)

In [12]:
user_df = user_df[~user_df.isin(['NULL']).any(axis=1)]
user_df.describe()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
count,15299,15299,15299,15299,15299,15299,15299,15299,15299,15299,15299
unique,2177,1191,11359,12104,15299,15299,18,19,15091,8267,15299
top,Michael,Smith,1958-02-02,Smith Inc,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,United Kingdom,GB,+44(0)808 1570796,2006-05-30,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
freq,57,340,6,22,1,1,9371,9365,3,8,1


In [1]:
def upload_to_db(df, table_name):
        DATABASE_TYPE = 'postgresql'
        DBAPI = 'psycopg2'
        HOST = 'localhost'
        USER = 'postgres'
        PASSWORD = 'JunkDB'
        DATABASE = df
        PORT = 5432
        try:
            engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")# Create a database engine using SQLAlchemy
            df.to_sql(table_name, engine, if_exists='replace', index=False)
            print(f"Data has been uploaded to the '{table_name}' table successfully.")
    
        except Exception as e:
            print(f"An error occurred: {str(e)}")